Doing transformation for all tables

In [0]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql import SparkSession

table_name = []
for i in dbutils.fs.ls('/mnt/silver/SalesLT'):
    table_name.append(i.name.split('/')[0])

In [0]:
table_name

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
for i in table_name:
    path = f'/mnt/silver/SalesLT/{i}'
    
    if dbutils.fs.ls(path):
        df = spark.read.format('delta').load(path)
        column_names = df.columns

        for old_col_name in column_names:
            # Convert column name from ColumnName to Column_Name format
            new_col_name = "".join(["_" + char if char.isupper() and not old_col_name[i-1].isupper() else char for i, char in enumerate(old_col_name)]).lstrip("_")

            # Change the column name using withColumnRenamed and regexp_replace
            df = df.withColumnRenamed(old_col_name, new_col_name)
        
        output_path = f'/mnt/gold/SalesLT/{i}/'
        df.write.format('delta').mode("overwrite").save(output_path)